# L3: Advanced queries with Agents

In this lesson, you are going to do more advanced querying.

### Import required dependencies and load the API key

> **Note:** This code was updated in January 2025 to upgrade from `llamaindex` 0.1.18 to 0.8.30. The older version relied on `gpt-3.5-turbo-0613`, which has since been deprecated.

**Different Run Results:** The output generated by AI chat models can vary with each execution due to their probabilistic nature. Don't be surprised if your results differ from those shown in the video.

In [16]:
import { 
    Document, 
    VectorStoreIndex, 
    SimpleDirectoryReader,
    RouterQueryEngine,
    OpenAIAgent,
    QueryEngineTool,
    FunctionTool,
} from "npm:llamaindex@0.8.30"

Let's create 2 different data sources that answer different types of questions:
- The first is the same example as in lesson 1, an essay by Dan Abramov about his career.
- The second is the wikipedia page content about React.

**Note**: To access the data used in this lesson, go to `File` and click on `Open` 

In [2]:
const documents1 = await new SimpleDirectoryReader().loadData({directoryPath: "./data"})
const index1 = await VectorStoreIndex.fromDocuments(documents1)
const queryEngine1 = index1.asQueryEngine()

In [3]:
let response1 = await queryEngine1.query({query: "What did the author do in college?"})
console.log(response1.toString())

In college, the author initially attended a program that did not have much programming in the curriculum for the first two years, focusing instead on subjects like linear algebra and physics, which the author did not find particularly interesting. The author started slacking off, skipping lectures, and eventually stopped attending college altogether after getting a job as a software developer. The author did not show up for midterm exams and returned the MacBook provided by the college, only returning five years later to pick up their papers.


In [4]:
const documents2 = await new SimpleDirectoryReader().loadData({directoryPath: "./data2"})
const index2 = await VectorStoreIndex.fromDocuments(documents2)
const queryEngine2 = index2.asQueryEngine()

In [5]:
let response2 = await queryEngine2.query({query:"What is React?"})
console.log(response2.toString())

React, also known as React.js or ReactJS, is a free and open-source front-end JavaScript library used for building user interfaces based on components. It is maintained by Meta (formerly Facebook) and a community of individual developers and companies. React is primarily used to develop single-page, mobile, or server-rendered applications, often in conjunction with frameworks like Next.js. It focuses on rendering components to the Document Object Model (DOM) and typically relies on other JavaScript libraries for routing and additional client-side functionality.


### Create a router query engine

In [6]:
const queryEngine = await RouterQueryEngine.fromDefaults({
  queryEngineTools: [
    {
      queryEngine: queryEngine1,
      description: "Useful for questions about Dan Abramov",
    },
    {
      queryEngine: queryEngine2,
      description: "Useful for questions about the React library",
    },
  ],
});

### Ask it a question

In [7]:
let response3 = await queryEngine.query({query: "What is React?"})
console.log(response3.toString())

React, also known as React.js or ReactJS, is a free and open-source front-end JavaScript library used for building user interfaces based on components. It is maintained by Meta (formerly Facebook) and a community of individual developers and companies. React is primarily used to develop single-page, mobile, or server-rendered applications, often in conjunction with frameworks like Next.js. It focuses on rendering components to the Document Object Model (DOM) and typically relies on other JavaScript libraries for routing and additional client-side functionality.


In [8]:
let response4 = await queryEngine.query({query: "What did Dan Abramov do in college?"})
console.log(response4.toString())

Dan Abramov started college in Russia, where he initially hoped to study programming. However, his math exam scores were not high enough to get into his preferred college, so he chose a college that provided Macbooks to students. During his first year, he found the curriculum lacking in programming content and was instead filled with subjects like linear algebra and physics, which he did not find particularly interesting. He began to slack off, skip lectures, and eventually stopped attending college altogether. Despite being a lousy student, he got his first job as a software developer during the summer of 2010, which led him to lose motivation for college. He eventually dropped out and returned his Macbook, only going back to the college five years later to pick up his papers.


### Define a simple function

In [9]:
function sumNumbers({a,b}) {
  return a + b;
}

### Explain the function in a JSON structure

In [10]:
const sumJSON = {
  type: "object",
  properties: {
    a: {
      type: "number",
      description: "The first number",
    },
    b: {
      type: "number",
      description: "The second number",
    },
  },
  required: ["a", "b"],
};

### Make the new function into a tool

In [11]:
const sumFunctionTool = new FunctionTool(sumNumbers, {
  name: "sumNumbers",
  description: "Use this function to sum two numbers",
  parameters: sumJSON,
});

### Make the router query engine into a tool

In [12]:
const queryEngineTool = new QueryEngineTool({
    queryEngine: queryEngine,
    metadata: {
        name: "react_and_dan_abramov_engine",
        description: "A tool that can answer questions about Dan Abramov and React",
    },
});

### Use the two tools

In [13]:
const agent = new OpenAIAgent({
    tools: [queryEngineTool, sumFunctionTool],
    verbose: true
})

### Let's ask it a question!

In [14]:
let response5 = await agent.chat({message:"What is React? Use a tool."})
console.log(response5.toString())

React, also known as React.js or ReactJS, is a free and open-source front-end JavaScript library used for building user interfaces based on components. It is maintained by Meta (formerly Facebook) and a community of individual developers and companies. React is primarily used to develop single-page, mobile, or server-rendered applications, often in conjunction with frameworks like Next.js. It focuses on rendering components to the Document Object Model (DOM) and typically relies on additional libraries for routing and other client-side functionalities.


In [15]:
let response6 = await agent.chat({message:"What is 501 + 5?"})
console.log(response6.toString())

501 + 5 equals 506.
